In [ ]:
library(ropls)
library(ggsci)
library(Cairo)
library(tidyverse)
library(malbacR)
library(pmartR)
library(ggplot2)
library(dplyr)
library(tibble)
library(tidyr)
library(paletteer)
library(extrafont)

#### 设置分组标准

In [ ]:
group_standard <- 'VFA_group1'
save_file_prex <- 'wilcox_VFA_group1'
path_save <- paste('/mnt/data3/data_exp_0826/result/Development_Cohort_',save_file_prex,sep = '')
dir.create(path_save,showWarnings = FALSE)
path_save

#### 读取原始数据

In [ ]:
data_raw <- readRDS('/mnt/data3/data_exp_0826/result/amide_qcrfsc.rds')

In [ ]:
options(repr.matrix.max.rows = 20,repr.matrix.max.cols = 20)
data_raw$e_data

In [ ]:
pmart_amide_norm <- data_raw

In [ ]:
sampleMetadata <- pmart_amide_norm$f_data
sampleMetadata %>% head()

In [ ]:
sampleMetadata$Sample %>% duplicated() %>% length()

In [ ]:
sampleMetadata$group_lv1 %>% unique()
sampleMetadata$group_lv2 %>% unique()

In [ ]:
sampleMetadata_use <- sampleMetadata %>%
  filter(group_lv1 %in% c('Development Cohort'),group_lv2 == 'time_1')

In [ ]:
sample_select <- sampleMetadata_use %>%
  pull(Sample) %>% unique()
sample_select %>% length()
sample_select

In [ ]:
dataMatrix_use <- pmart_amide_norm$e_data %>% 
  column_to_rownames('Molecule') %>% 
  t() %>% as.data.frame() %>% 
  rownames_to_column('Sample') %>% 
  filter(Sample %in% sample_select) %>% 
  column_to_rownames('Sample')
dataMatrix_use %>% dim()
dataMatrix_use %>% head()

In [ ]:
group_use <- sampleMetadata_use %>% 
  mutate(Sample = factor(Sample,levels = sample_select)) %>% 
  arrange(Sample) %>% 
  pull(all_of(group_standard))
group_use %>% table()

In [ ]:
variableMetadata_use <- pmart_amide_norm$e_meta
variableMetadata_use

In [ ]:
options(repr.plot.width = 20,repr.plot.height = 20)
pca = opls(dataMatrix_use)

In [ ]:
dataMatrix_use %>% max()
dataMatrix_use %>% min()
group_use

In [ ]:
options(repr.plot.width = 20,repr.plot.height = 20)
results_plsda <- list()
for (i in 1:10) {
  results_plsda[[i]] <- opls(dataMatrix_use, group_use, predI = i, orthoI = 0)
  print(paste("orthoI =", i))
  print(results_plsda[[i]]@summaryDF)
}

best_model <- results_plsda[[which.max(sapply(results_plsda, function(x) x@summaryDF$`Q2(cum)`))]]


In [ ]:
lapply(1:10,function(i){results_plsda[[i]]@summaryDF})

In [ ]:
plsda = opls(dataMatrix_use,group_use,predI = 10,orthoI = 0)


In [ ]:
# sample scores plot
sample.score = plsda@scoreMN %>% 
  as.data.frame() %>%
  mutate(group_use = group_use)
sample.score %>% head()

In [ ]:
options(repr.plot.height = 10,repr.plot.width = 12)
ggplot(sample.score, aes(p1, p2, color = group_use)) +
  geom_hline(yintercept = 0, linetype = 'dashed', size = 0.5) +
  geom_vline(xintercept = 0, linetype = 'dashed', size = 0.5) +
  geom_point(size = 5) +
  # geom_point(aes(-10,-10), color = 'white') +
  annotate("point", x = -10, y = -10, color = "white") +
  labs(x = 'P1(10.0%)',y = 'P2(9%)') +
  stat_ellipse(level = 0.95, linetype = 'solid', 
               linewidth = 1, show.legend = FALSE) +
  scale_color_manual(values = c('#008000','#FFA74F')) +
  guides(color = guide_legend(override.aes = list(size = 5))) +
  theme_bw() +
  theme(legend.position = c(0.15,0.9),
    legend.text = element_text(color = 'black',size = 20, family = 'Arial', face = 'plain'),
    legend.title = element_text(size = 24),
    panel.background = element_blank(),
    panel.grid = element_blank(),
    axis.text = element_text(color = 'black',size = 15, family = 'Arial', face = 'plain'),
    axis.title = element_text(color = 'black',size = 15, family = 'Arial', face = 'plain'),
    axis.ticks = element_line(color = 'black'))

In [ ]:
vip.score = as.data.frame(plsda@vipVn)
colnames(vip.score) = 'vip'
vip.score$metabolites = rownames(vip.score)
vip.score = vip.score[order(-vip.score$vip),]
vip.score$metabolites = factor(vip.score$metabolites,
                               levels = vip.score$metabolites)
 
loading.score = plsda@loadingMN %>% as.data.frame()
loading.score$metabolites = rownames(loading.score)
 
all.score = merge(vip.score, loading.score, by = 'metabolites')
 
all.score$cat = paste('A',1:nrow(all.score), sep = '')

In [ ]:
options(repr.plot.width = 15,repr.plot.height = 6)
ggplot(all.score[all.score$vip >= 1,], aes(cat, vip)) +
  geom_segment(aes(x = cat, xend = cat,
                   y = 0, yend = vip)) +
  geom_point(shape = 21, size = 5, color = '#008000' ,fill = '#008000') +
  annotate(geom = 'point',x=1,y=2.5, color = 'white') +
  geom_hline(yintercept = 1, linetype = 'dashed') +
  scale_y_continuous(expand = c(0,0)) +
  labs(x = '', y = 'VIP value') +
  theme_bw() +
  theme(legend.position = 'none',
        legend.text = element_text(color = 'black',size = 12, family = 'Arial', face = 'plain'),
        panel.background = element_blank(),
        panel.grid = element_blank(),
        axis.text = element_text(color = 'black',size = 15, family = 'Arial', face = 'plain'),
        axis.text.x = element_text(angle = 90),
        axis.title = element_text(color = 'black',size = 15, family = 'Arial', face = 'plain'),
        axis.ticks = element_line(color = 'black'),
        axis.ticks.x = element_blank())

#### oplsda

In [ ]:
options(repr.plot.width = 20,repr.plot.height = 20)
results <- list()
for (i in 0:7) {
  results[[i+1]] <- opls(dataMatrix_use, group_use, predI = 1, orthoI = i)
  print(paste("orthoI =", i))
  print(results[[i+1]]@summaryDF)
}

best_model <- results[[which.max(sapply(results, function(x) x@summaryDF$`Q2(cum)`))]]


In [ ]:
# OPLS-DA analysis
options(repr.plot.width = 15,repr.plot.height = 15)
dataMatrix_use %>% max()
dataMatrix_use %>% min()
oplsda = opls(dataMatrix_use, group_use, predI = 1, orthoI = 1)#,permI = 200

In [ ]:
sample.score = oplsda@scoreMN %>% 
  as.data.frame() %>%
  mutate(group_use = group_use,
         o1 = oplsda@orthoScoreMN[,1])

In [ ]:
sample.score

In [ ]:
options(repr.plot.width = 10,repr.plot.height = 8)
ggplot(sample.score, aes(p1, o1, color = group_use)) +
  geom_hline(yintercept = 0, linetype = 'dashed', size = 0.5) +
  geom_vline(xintercept = 0, linetype = 'dashed', size = 0.5) +
  geom_point(size = 2) +
  #geom_point(aes(-10,-10), color = 'white') +
  labs(x = 'P1(5.0%)',y = 'to1') +
  stat_ellipse(level = 0.95, linetype = 'solid', 
               size = 1, show.legend = FALSE) +
  scale_color_manual(values = c('#008000','#FFA74F')) +
  guides(color = guide_legend(override.aes = list(size = 5))) +
  theme_bw() +
  theme(legend.position = c(0.1,0.95),
        legend.title = element_blank(),
        legend.text = element_text(color = 'black',size = 12, family = 'Arial', face = 'plain'),
        panel.background = element_blank(),
        panel.grid = element_blank(),
        axis.text = element_text(color = 'black',size = 15, family = 'Arial', face = 'plain'),
        axis.title = element_text(color = 'black',size = 15, family = 'Arial', face = 'plain'),
        axis.ticks = element_line(color = 'black'))

In [ ]:
# VIP scores plot
vip.score = as.data.frame(oplsda@vipVn)
colnames(vip.score) = 'vip'
vip.score$metabolites = rownames(vip.score)
vip.score = vip.score[order(-vip.score$vip),]
vip.score$metabolites = factor(vip.score$metabolites,
                               levels = vip.score$metabolites)
vip.score %>% head()

In [ ]:
pvaVn <- apply(dataMatrix_use, 2,
               function(feaVn) cor.test(group_use %>% as.factor() %>% as.numeric(), feaVn)[["p.value"]])

vipVn <- getVipVn(opls(dataMatrix_use, group_use,
                       predI = 1, orthoI = 1,
                       fig.pdfC = "none"))
vipVn %>% head()

In [ ]:
# pvaVn <- apply(dataMatrix_use, 2,
#                function(feaVn) cor.test(group_use %>% as.factor() %>% as.numeric(), feaVn)[["p.value"]])
# p.adj = p.adjust(pvaVn, method = 'BH')
e_meta <- pmart_amide_norm$e_meta %>% as.data.frame() %>% 
    dplyr::rename(metabolites = Molecule)
loading.score = oplsda@loadingMN %>% as.data.frame()
loading.score$metabolites = rownames(loading.score)
all.score  <-  merge(vip.score, loading.score, by = 'metabolites') %>% 
    # mutate(pvalue = pvaVn,p.adj = p.adj) %>% 
    left_join(e_meta ,by = 'metabolites')
all.score$cat = paste('A',1:nrow(all.score), sep = '')
all.score %>% dim()
all.score %>% head()

In [ ]:
library(stringi)
data_plot <- all.score %>% 
    filter(vip >= 1) %>% 
    arrange(desc(vip)) %>% 
    dplyr::slice(1:12,14:21) %>% 
    mutate(metabolites = stri_enc_toutf8(metabolites)) %>%
    mutate(
        metabolites = metabolites %>% str_wrap(width = 50,whitespace_only = FALSE)
    ) %>% 
    mutate(metabolites = factor(metabolites,levels = metabolites %>% rev()))
data_plot %>% dim()
data_plot$level  %>% table()
data_plot %>% head()

In [ ]:
options(repr.plot.width = 14,repr.plot.height = 15)
ggplot(data_plot, aes(metabolites, vip)) +
  geom_segment(aes(x = metabolites, xend = metabolites,
                   y = 0, yend = vip)) +
  geom_point(shape = 21, size = 5, color = '#008000' ,fill = '#008000') +
  annotate(geom = 'point',x = 1,y = 2.5, color = 'white') +
  geom_hline(yintercept = 1, linetype = 'dashed') +
  scale_y_continuous(expand = c(0,0),limits = c(0,3.5)) +
  labs(x = '', y = 'VIP value') +
  coord_flip() +
  theme_bw() +
  theme(legend.position = 'none',
        legend.text = element_text(color = 'black',size = 12, family = 'Arial', face = 'plain'),
        panel.background = element_blank(),
        panel.grid = element_blank(),
        axis.text = element_text(color = 'black',size = 30, family = 'Arial', face = 'plain'),
        axis.text.x = element_text(angle = 0,hjust = 1),
        axis.title = element_text(color = 'black',size = 20, family = 'Arial', face = 'plain'),
        axis.ticks = element_line(color = 'black'),
        axis.ticks.x = element_blank())

In [ ]:
# volcano plot
df = dataMatrix_use %>% as.data.frame()
df$group_use = group_use
df = df[order(df$group_use),]
df = df %>% select(-c('group_use'))
df

In [ ]:
sample_select_m <- sampleMetadata_use$Sample[sampleMetadata_use[[group_standard]] == 'sarcopenia']
sample_select_m_rowid <- match(sample_select_m,rownames(df))
sample_select_f <- sampleMetadata_use$Sample[sampleMetadata_use[[group_standard]] == 'non-sarcopenia']
sample_select_f_rowid <- match(sample_select_f,rownames(df))
sample_select_m_rowid;sample_select_f_rowid

In [ ]:
M.mean = apply(df[sample_select_m_rowid,],2,FUN = mean)
F.mean = apply(df[sample_select_f_rowid,],2,FUN = mean)
FC = M.mean - F.mean
log2FC = FC
# pvalue = apply(df, 2, function(x)
#   {t.test(x[sample_select_m_rowid],x[sample_select_f_rowid])$p.value})
pvalue = apply(df, 2, function(x)
 {wilcox.test(x[sample_select_m_rowid], x[sample_select_f_rowid])$p.value})

p.adj = p.adjust(pvalue, method = 'BH')
p.adj.log = -log10(p.adj)
colcano.df = data.frame(FC,log2FC,pvalue,p.adj, p.adj.log)
colcano.df$cat = ifelse(colcano.df$log2FC >= log2(1.2) & colcano.df$pvalue < 0.05,'Up',
                        ifelse(colcano.df$log2FC <= log2(1/1.2) & colcano.df$pvalue < 0.05,'Down','NS'))
colcano.df$cat %>% table()
colcano.df %>% head()

In [ ]:
e_meta <- pmart_amide_norm$e_meta %>% as.data.frame() %>% 
    dplyr::rename(metabolites = Molecule)
loading.score = oplsda@loadingMN %>% as.data.frame()
loading.score$metabolites = rownames(loading.score)
all.score  <-  merge(vip.score, loading.score, by = 'metabolites') %>% 
    # mutate(pvalue = pvaVn,p.adj = p.adj) %>% 
    left_join(e_meta ,by = 'metabolites')
all.score$cat = paste('A',1:nrow(all.score), sep = '')
all.score %>% dim()
all.score %>% head()

In [ ]:
options(repr.matrix.max.cols = 40)
df_res  <-  colcano.df %>% 
    rownames_to_column('metabolites') %>% 
    left_join(all.score %>% dplyr::select(-c('cat')),by = 'metabolites') %>% 
    dplyr::select(c('HMDB','metabolites','Super.Class','Class','Sub.Class','level',
              'log2FC','pvalue','p.adj','vip','p1','cat','Annotation','Reactome_id','Reactome_Description')) %>% 
    arrange(log2FC) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(cat,desc(log2FC)) %>% 
    as.data.frame()
file_path <- paste(path_save,'/df_res_baseline_sarcopenia vs non-sarcopenia.csv',sep = '')
cat(file_path)
write.csv(df_res,file_path)
df_res$cat %>% table()
df_res %>% head()

In [ ]:
options(repr.matrix.max.cols = 40)
df_res  <-  colcano.df %>% 
    rownames_to_column('metabolites') %>% 
    left_join(all.score %>% select(-c('cat')),by = 'metabolites') %>% 
    arrange(log2FC) %>% 
    mutate(
        cat = case_when(
            vip > 1 & log2FC >= log2(1.2) & pvalue < 0.05 ~ 'Up',
            vip > 1 & log2FC < log2(1/1.2) & pvalue < 0.05 ~ 'Down',
            vip > 1 & pvalue > 0.05~ 'vip_sig but p_value_NS',
            TRUE ~ 'NS'
        ),
        cat = factor(cat,levels = c('Up','Down','vip_sig but p_value_NS','NS'))
    ) %>% 
    arrange(cat,desc(log2FC)) %>% 
    as.data.frame()
file_path <- paste(path_save,'/df_res_baseline_sarcopenia vs non-sarcopenia_raw.csv',sep = '')
cat(file_path)
write.csv(df_res,file_path)